<a href="https://colab.research.google.com/github/alihammadbaig/pdf_to_excel/blob/master/pdf_to_excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!ls "/content/gdrive/My Drive/data"

Mall_Customers.csv  pdf_to_excel.pdf  pdf_to_excel.txt


In [3]:
pip install tika

  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
Successfully built tika


In [0]:
# Text parsing library
import tika
from tika import parser

# Deal with regular expression(text processing)
import re

# Handle the excel spreadsheet
import openpyxl

import os

tika.initVM()

# Read the ;PDF file
parsed = parser.from_file('/content/gdrive/My Drive/Colab Notebooks/data/sample_pdf_for_excel.pdf')
content = parsed["content"].strip()


In [60]:
# Read the spreadsheet for later use
wb = openpyxl.load_workbook("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
sheet = wb['Template']
# Starting writing the sheet beginning line#2
sheet_row_num = 2

# PDF columns
col_names = ["top_coat_date", "blast_primer_date"
             ,"dft1_ave", "dft1_5", "dft1_4", "dft1_3", "dft1_2", "dft1_1"
             ,"dft2_ave", "dft2_5", "dft2_4", "dft2_3", "dft2_2", "dft2_1"
             , "sys_tem"]

# Spreadsheet cells
cells = ["AL", "S", "AC", "AB", "AA", "Z", "Y", "X", "K", "J", "I", "H", "G", "F", "D"]

cols = {}

# c = content
page_num_printed = 5479
page_num = 1
skip = 11
total_pages = int(parsed["metadata"]["xmpTPg:NPages"].strip())+1

def get_col():
  global r
  r = r.strip()
  val = r[r.rfind(" ") : len(r)]
  r = r[0 : r.rfind(" ")]
  return val

def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))


BadZipFile: ignored

In [56]:
'''
## Pages ##
'''
for page_num in range(7, total_pages):
  print("Page#%i"%page_num)
  print("Page num printed#%i"%page_num_printed)
  
  start_of_page = "Page " + str(page_num_printed) +" of 13699"
  end_of_page = "Page " + str(page_num_printed + 1) +" of 13699"
  page_content = content[content.find(start_of_page): content.find(end_of_page)]
  
  
  '''
  ## Extract the table
  '''
  try:
    p = re.compile("(A*VE\\s+DATE)", re.IGNORECASE)
    for m in p.finditer(page_content):
      start_of_table = m.start()+8

    end_of_table = re.search("(FINAL\\s+)*INSPECTION", page_content, flags = re.I)
    if end_of_table is None:
      end_of_table = re.search("(VISUAL\\s+)*INSPECTION", page_content, flags = re.I)
      skip = 29
    
    if end_of_table is None: 
      end_of_table = re.search("\\(*REMARK", page_content, flags = re.I)
      skip = 50
    
    end_of_table = end_of_table.start() - skip
    
    table = page_content[start_of_table:end_of_table]

    # Data cleansing
    table = table.replace("·","-")\
      .replace("•","")\
      .replace(" ' "," ")\
      .replace("l4", "14")\
      .replace(",", "")\
      .replace(" . ", " ")\
      .replace(" - ", " ")\
    
    table = re.sub("OS-([A-Z]{3})", r"05-\1", table,  flags=re.I)

    page_num_printed = page_num_printed + 1


    '''
    ## Rows ##
    '''
    for row in [x for x in table[2:].splitlines() if x]:
      row = row.strip()
      row = re.sub("-\\s*$", "", row)
      row = re.sub("^.*?\\s+", "", row)
      row = re.sub("^\\s*\\w+\\s+", "", row)
      
      if len(row) > 50  and\
        hasNumbers(row):
        
        # Check if the second last columna is a date columns or not
        # In some cases the last date column is not read
        row_cols = row.split(' ')
        second_last_col = row_cols[len(row_cols)-1].strip()
        
        if not bool(re.match("\d{2}\-[a-z]{3}\-\d{4}", second_last_col, flags = re.I)):
          # append the available date column at the end of the row
          row = row + " " + second_last_col
          print("Last date column missing. adding second last column: %s"%second_last_col)
          print(row)
        
        global r
        r = row
          
        for col_name in col_names:
          col_val = get_col()
          cols[col_name] = col_val
          col_val = re.sub(r"([0-9]+)_", "\g<1>",col_val)
          col_val = re.sub(r"_([0-9]+)", "\g<1>",col_val)
          col_val = re.sub(r"\s+-([0-9]+)", " \g<1>",col_val)
          col_val = re.sub(r"([0-9]+)-\s+", "\g<1> ",col_val)
          
          # Write in spreadsheet
          i = 0
          cell = cells[i] + str(sheet_row_num)    
          sheet[cell].value= col_val
          i = i + 1

        r = re.sub("\\s+", "_", r)
        cols["spool_no"] = r
        cell ="D"+str(sheet_row_num)    
        sheet[cell].value = r

        cell ="C"+str(sheet_row_num)    
        sheet[cell].value = page_num
      
        sheet_row_num = sheet_row_num + 1

      wb.save("/content/gdrive/My Drive/Colab Notebooks/data/excel_instructions.xlsx")
  except Exception:
    print("Exception on page%i" %page_num)
    traceback.print_exc()
    pass
    


Page#7
Page num printed#5485
Last date column missing. adding second last column: 06-Nov-14
IA2EC-692 "'1nno.1ro1.~<-n1 1 150 110 133 148 122 133 204 243 217 256 230 230 06-Nov-14 06-Nov-14
Last date column missing. adding second last column: 06-Nov-14
43032 I A1EN-561 1 139 102 129 140 119 126 _2<l! _2~7 221 260 234 234 06-Nov-14 06-Nov-14
Last date column missing. adding second last column: 06-Nov-14
l c7EA 10 or:n<.l.>~ "-"' 1 142 106 125 144 115 126 ~ ~ ~ ~ ~ ~ 06-Nov-14 06-Nov-14


KeyboardInterrupt: ignored